# Prática 2: análise e representação de dados coletados por flutuadores Deep BCG Argo

Hoje você manipulará dados de temperatura, salinidade e oxigênio coletados por flutuadores biogeoquímicos Deep Argo no Atlântico. Trabalhar com esses dados requer aprender um pouco sobre arquivos netCDF. É importante não se distrair com a complexidade do conjunto de dados. Trabalhar com arquivos netCDF e perfis Argo se tornará muito mais fácil quando aprendermos sobre xarray em duas semanas. Por enquanto, no entanto, vamos focar nas ferramentas que já aprendemos.

Instruções:

Crie os melhores gráficos possíveis, prestando atenção especial a rótulos, unidades, mapas de cores, etc.
Salve as suas figuras em formato .png.
Descreva os resultados: o que você vê?
Discuta os resultados: o que explica as diferenças entre os perfis?
Se você estiver confortável com loops "for", tente usar o mínimo possível de código repetitivo.






### Antes de começar
Módulos utilizados na prática 2. É sempre um bom hábito carregar os módulos que você irá usar no início do notebook)

In [ ]:
#
# Módulos básicos do Python
#

import numpy as np                     # numpy para manipulação de arrays
from netCDF4 import Dataset            # netCDF para leitura de arquivos netCDF
import matplotlib.pyplot as plt        # bilioteca de plotagem


#
# Pacotes especializados
#

import gsw                             # Toolbox Gibbs-SeaWater para cáculo envolvendo a equação de estado da água do mar
import cmocean                         # Colormaps oceanográficos

### Step 1:  Passo 1: Aprendendo o básico de como carregar um arquivo netCDF no Python

In [ ]:
ls /data/bcg-argo/

In [ ]:
data_dir =  '/data/bcg-argo/'               # Define data_dir for convenience
filename = 'SR6902974_045.nc'            
SR6902974 = Dataset(data_dir + filename) # load 

In [ ]:
# Se você digitar o nome do conjunto de dados netCDF, verá seus metadados
SR6902974

In [ ]:
# Verifique apenas as "chaves" (nomes das variáveis):
SR6902974.variables.keys()

In [ ]:
# São muitas variáveis, incluindo informações sobre o tipo de dado, formato,
# posição, horário do perfil e vários dados sobre o processo de controle de qualidade

# Para os metadados de uma única variável, por exemplo, TEMP:
SR6902974['TEMP']

In [ ]:
# Outro exemplo é a variável pressão (pressure)
SR6902974['PRES']

A coordenada vertical é a pressão, que é medida por um sensor de quartzo no CTD. A pressão da água do mar é relatada em dbar (decibar), onde 1 dbar = 0,1 bar = 10000 Pa (Pascal). Os oceanógrafos gostam de dbar porque uma coluna de 1 metro de água do mar é, para uma excelente aproximação, equivalente a 1 dbar.

In [ ]:
# Finalmente, também estaremos trabalhando com oxigênio dissolvido hoje:
SR6902974['DOXY']

In [ ]:
# Você também pode acessar a posição e o horário em que o perfil foi coletado
SR6902974['LONGITUDE'][0].data, SR6902974['LATITUDE'][0].data, SR6902974['JULD'][0].data

O tempo JULD acima é um dia juliano, medindo o tempo decorrido (em) dias desde um tempo de referência. Para flutuadores Argo, o tempo de referência é 01/01/1950 às 0H:0M:0S GMT (também conhecido como 000).

In [ ]:
# Esses dados de referência são registrados no arquivo netCDF:
ref_date = SR6902974['REFERENCE_DATE_TIME'][:].data
''.join(list(map(lambda x: x.decode('utf-8'), ref_date))).strip()  # Don't get scared by this cryptic line of code: it simply prints the reference date in YYYYMMDDMMSSmm

In [ ]:
# Você pode plotar um perfil de temperature, por exemplo:
plt.plot(SR6902974['TEMP'][0],SR6902974['PRES'][0])
plt.ylim(3500,0)
plt.xlabel(SR6902974['TEMP'].long_name + ' ['+ SR6902974['TEMP'].units + ']')
plt.ylabel(SR6902974['PRES'].long_name[:18] + ' ['+ SR6902974['PRES'].units + ']')

### Passo 2.1: Plote a salinidade (PSAL) e o oxigênio dissolvido (DOXY). Você percebe algo estranho com esses perfis? Discuta os resultados.

Note que DOXY possui algumas lacunas que aparecem como valores mascarados, então você deve usar um marcador ("." ou "o" ou "-o") para plotá-lo. Veja este https://matplotlib.org/stable/gallery/lines_bars_and_markers/masked_demo.html para alternativas.

In [ ]:
fig = plt.figure(figsize=(10,4.5))

ax1 = fig.add_subplot(121)
plt.plot(SR6902974['PSAL'][0],SR6902974['PRES'][0],'.')
plt.ylim(3500,0)
plt.xlabel(SR6902974['PSAL'].long_name + ' ['+ SR6902974['PSAL'].units + ']')
plt.ylabel(SR6902974['PRES'].long_name[:18] + ' ['+ SR6902974['PRES'].units + ']')

fig.subplots_adjust(wspace=.4)

ax2 = fig.add_subplot(122)
plt.plot(SR6902974['DOXY'][0],SR6902974['PRES'][0],'.-')
plt.ylim(3500,0)
plt.xlabel(SR6902974['DOXY'].long_name + ' ['+ SR6902974['DOXY'].units + ']')
plt.ylabel(SR6902974['PRES'].long_name[:18] + ' ['+ SR6902974['PRES'].units + ']')

# !mkdir img
plt.savefig('img/Salinity_and_Oxygen_Profiles.png',dpi=100)

### Passo 2: Carregando todos os dados

Para evitar confusão nesta prática, vamos colocar as informações mais relevantes em dicionários. Abaixo estão duas funções em python que o César escreveu para esta tarefa.

In [ ]:
def jstring(bstring):
    """" Join an array of bstrings into a single string """
    return ''.join(list(map(lambda bstring: bstring.decode('utf-8'), bstring))).strip()

def netCDFtoDict(dataset): 
    """ This function puts the most basic information of an Argo profile
         into a dictionary for simple manipulation. 
         
         Written for an in-class assignment of MARN-5895, UConn.
    
    Input:
    
        - a netCDF Dataset with the Argo profile
        
    Output:
    
        - a dictionary with the profile data, including
        
                - platform (platform number), 
                - longitude (single float number),
                - latitude (single float number),
                - pressure (array of pressure in dbar)
                - temperature (array of temperature in degC)
                - salinity (array of salinity in psu)
                - doxygen (array of dissolved oxygen in micro mole per kg)
         
    """
    
    # mask gaps
    doxy = dataset['DOXY_ADJUSTED'][0].data
    doxy = np.ma.masked_where(doxy==99999.,doxy)
    
    profile_dict = dict(platform=jstring(dataset['PLATFORM_NUMBER'][0].data),
                        longitude=dataset['LONGITUDE'][0].data,
                        latitude=dataset['LATITUDE'][0].data,
                        pressure=dataset['PRES_ADJUSTED'][0].data,
                        temperature=dataset['TEMP_ADJUSTED'][0].data,
                        salinity=dataset['PSAL_ADJUSTED'][0].data,
                        doxygen=doxy,
    )
    
    return profile_dict

In [ ]:
# Exemplo de uso da função:
filename = 'SR6902974_045.nc'            
SD6903573 = netCDFtoDict(Dataset(data_dir + filename))

In [ ]:
# Agora podemos usar isso para todos os arquivos
# Primeiro, usamos o módulo glob para obter o caminho+nome de todos os arquivos de dados em data_dir
import glob
filenames = glob.glob(data_dir+'/*.nc')
print(filenames)

In [ ]:
#Para obter o número de ID do flutuador, podemos fatiar a string do nome do arquivo, por exemplo, para obter o ID do flutuador do elemento zero de filenames:
filenames[0][:-3]

In [ ]:
# Colocando tudo junto: percorra os nomes dos arquivos, carregue o conjunto de dados netCDF e coloque as informações básicas em um dicionário
floats = []
for filename in filenames:
    dname = filename[15:-7]
    print('Generating dictionary '+ dname)
    dataset=Dataset(filename)
    exec(dname + ' = netCDFtoDict(dataset)')
    floats.append(dname)

In [ ]:
#  A lista "floats" contém o nome/ID de cada flutuador
# Isso pode ser útil no seu código de plotagem abaixo
floats

In [ ]:
# O comando "eval" é útil: ele recebe uma string
# como argumento e a transforma em um comando/variável Python
argo_float = eval('SR6902892')
argo_float['temperature']

In [ ]:
#Esta célula gera um mapa rápido e simples
# com as posições dos flutuadores.
#(Iremos aprender sobre o cartopy em algumas semanas.)

import cartopy.crs as ccrs

# plot quick map with positions
plt.figure(figsize=(8,8))
ax = plt.axes(projection=ccrs.PlateCarree())
ax.gridlines(crs=ccrs.PlateCarree(), draw_labels=True,
             linewidth=1, color='gray', alpha=0.5, linestyle='--')
ax.coastlines()

for profile in floats:
    
    argo=eval(profile)
    
    ax.plot(argo['longitude'],
            argo['latitude'],
            marker='o',
            markersize=7,
            color='k'
           )
    
    ax.text(argo['longitude']-1.0,
            argo['latitude']+1.5,
            profile,
            color='k',
            rotation=60,
            fontsize=8
           )
    
ax.set_xlim(-85,25)
ax.set_ylim(-70,80)

#### Antes de prosseguir, verifique se os dicionários foram criados.

## Passo 3: Cálculo da temperatura potencial e densidade potencial



3.1 Utilize a função gsw.pt_from_t para calcular a temperatura potencial a partir da temperatura in-situ para cada flutuador. Adicione os perfis de temperatura potencial aos dicionários, nomeando-os como ptemperature.

3.2 Utilize a função gsw.CT_from_pt para calcular a temperatura conservativa a partir da temperatura in-situ para cada flutuador. Adicione os perfis de temperatura conservativa aos dicionários, nomeando-os como ctemperature.

3.3 Utilize a função gsw.sigma0 para calcular a densidade potencial em relação à superfície (0 dbar). Adicione a densidade potencial aos dicionários, nomeando-a como sigma0. (Note que sigma é a densidade potencial menos um valor de referência 1000.)

Notas:

Lembre-se de acessar a ajuda da função usando um ponto de interrogação ?, por exemplo: gsw.sigma0?.
As funções acima requerem como entrada a quantidade de "Salinidade Absoluta" (SA), que em boa aproximação é a mesma que a salinidade calculada a partir da condutividade pelo CTD nos flutuadores Argo. Para os propósitos deste exercício, você pode usar salinity em vez de SA como entrada para as funções acima.





In [ ]:
for float_name in floats:
    
    print("Calculating derived fields for " + float_name)
    
    profile = eval(float_name)
    
    # 3.1
    profile['ptemperature'] = gsw.pt_from_t(profile['salinity'],
                                            profile['temperature'],
                                            profile['pressure'],
                                            p_ref=0)
    # 3.2
    profile['ctemperature'] = gsw.CT_from_pt(profile['salinity'],
                                            profile['ptemperature'])

    # 3.3
    profile['sigma0'] = gsw.sigma0(profile['salinity'],
                                   profile['ctemperature'])

### Passo 4: Perfis simples (propriedade vs. pressão)
Crie uma figura com quatro subplots contendo perfis de temperatura potencial (e compare com temperatura in-situ e conservativa), salinidade, densidade potencial e oxigênio dissolvido. Cada subplot deve conter seis perfis, um para cada flutuador. Observe que os perfis de oxigênio têm lacunas, portanto você pode precisar usar um marcador (".", "o" ou "-o") para plotá-lo. Veja https://matplotlib.org/stable/gallery/lines_bars_and_markers/masked_demo.html para alternativas.


In [ ]:
floats

### Passo 5: Perfis em função da latitude
Utilize a função plt.scatter para criar uma figura que mostre a dependência latitudinal do oxigênio dissolvido. Escolha um mapa de cores apropriado.

### Passo 6: Gráficos de propriedade-propriedade
4.1 Trace um diagrama theta-S, onde theta representa a temperatura potencial, com todos os seis perfis em um único gráfico. Adicione contornos de densidade ao seu diagrama, semelhante a esta figura.

4.2 Trace temperatura, salinidade e oxigênio dissolvido em função da densidade potencial.